### Generating training  data

In [1]:
import numpy as np
from random import shuffle

NUM_EXAMPLES = 2 ** 20

data_input = ['{0:020b}'.format(i) for i in range (NUM_EXAMPLES)]
shuffle(data_input)
data_input = [map(int, i) for i in data_input]
ti = []
for i in data_input:
    temp_list = []
    for j in i:
        temp_list.append([j])
    ti.append(np.array(temp_list)) 
data_input = ti    

data_output = []

for i in data_input:
    count = 0
    for j in i:
        if j[0] == 1:
            count+=1
    temp_list = ([0]*21)
    temp_list[count] = 1
    data_output.append(temp_list)    

### Generating validation data

In [2]:
SIZE_TRAINING = int(0.2 * NUM_EXAMPLES)
SIZE_VALIDATION_SET = int(0.1 * NUM_EXAMPLES)
print("# of training inputs {}".format(SIZE_TRAINING))
print("# of validation inputs {}".format(SIZE_VALIDATION_SET))

train_input = data_input[:SIZE_TRAINING]
train_output= data_output[:SIZE_TRAINING]

validation_input = data_input[SIZE_TRAINING: SIZE_TRAINING + SIZE_VALIDATION_SET]
validation_output= data_output[SIZE_TRAINING: SIZE_TRAINING + SIZE_VALIDATION_SET]

test_input = data_input[SIZE_TRAINING + SIZE_VALIDATION_SET:]
test_output = data_output[SIZE_TRAINING + SIZE_VALIDATION_SET:]
print("# of inputs {}".format(len(data_input)))
print ("# of training inputs {}".format(len(train_input)))
print ("# of validation inputs {}".format(len(validation_input)))
print ("# of test inputs {}".format(len(test_input)))

# of training inputs 209715
# of validation inputs 104857
# of inputs 1048576
# of training inputs 209715
# of validation inputs 104857
# of test inputs 734004


### Model

In [3]:
import tensorflow as tf

data = tf.placeholder(tf.float32, [None, 20, 1])
target = tf.placeholder(tf.float32, [None, 21])

num_hidden = 24
cell = tf.nn.rnn_cell.LSTMCell(num_hidden, state_is_tuple = True)

val, state = tf.nn.dynamic_rnn(cell, data, dtype = tf.float32)
val = tf.transpose(val, [1, 0, 2])
last = tf.gather(val, int(val.get_shape()[0]) - 1)

weight = tf.Variable(tf.truncated_normal([num_hidden, int(target.get_shape()[1])]))
bias = tf.Variable(tf.constant(0.1, shape=[target.get_shape()[1]]))
logits = tf.matmul(last, weight) + bias
prediction = tf.nn.softmax(logits)
cross_entropy = -tf.reduce_sum(target * tf.log(tf.clip_by_value(prediction, 1e010, 1.0)))

optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)

/Users/raman/anaconda/envs/ml/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


### Calclulation of Error

In [4]:
mistakes = tf.not_equal(tf.argmax(target, 1), tf.argmax(prediction, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

### Execution

In [5]:
from sklearn.utils import shuffle

init_op = tf.initialize_all_variables()
sess = tf.Session()

batch_size = 1000
n_batches = int(len(train_input)/batch_size)
n_epochs = 50

'''
def evaluate(X, Y):
    n_input = len(X)
    session = tf.get_default_session()
    total_accuracy = 0
    for offset in range(0, n_input, batch_size):
        batch_end = offset + BATCH_SIZE
        x_batch, y_batch = X_data[offset: batch_end], y_data[offset: batch_end]
        accuracy = session.run(accuracy_operation, feed_dict = {x: x_batch, y: y_batch})
        total_accuracy += (accuracy * len(x_batch))
    return total_accuracy/n_input
'''

with tf.Session() as sess:
    sess.run(init_op)    
    for i in range(n_epochs):
        #train_input, train_output = shuffle(train_input, train_output)
        for j in range(n_batches):
            begin = j * batch_size
            end = begin + batch_size
            batch_input, batch_output = train_input[begin: end], train_output[begin: end]
            sess.run(minimize, feed_dict = {data: batch_input, target: batch_output})
        if i is not 0 and i % 10 == 0:
            validation_error = sess.run(error, feed_dict = {data: validation_input, target: validation_output})
            print ("Epoch - {}, error {:3.1f}%".format(str(i), validation_error))
    print("Epoch {:2d} completed".format(i + 1))
    test_error = sess.run(error, feed_dict = {data: test_input, target: test_output}) 
    print("Epoch {:2d} error {:3.1f}%".format(i+1, 100 * test_error))


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch - 10, error 1.0%
Epoch - 20, error 1.0%
Epoch - 30, error 1.0%
Epoch - 40, error 1.0%
Epoch 50 completed
Epoch 50 error 99.9%
